## **Lab 2**

### **Overview**

This workshop demonstrates basic CUDA to SYCL conversion using SYCLomatic tool and compilation of the SYCL program using 
Intel® oneAPI DPC++/C++ Compiler.

You will learn about SYCL C++ code skeleton and construct.  

### **Introduction of SYCL™ C++**

To support data/compute parallelism in C++ whereby software programs have access to parallel computing resources in modern heterogenous system (combination of CPU, GPU, FPGA and accelerators), Khronos group has SYCL™ initiative which is an industry-driven standard body that adds data parallelism to C++ which is not fully supported by ISO C++ standard. **The objective of SYCL is to influence the direction of C++ standard for supporting heterogenous compute.**

A SYCL program is single-source and contains code sections (known as **host code** for orchestrating data movement and compute offload to devices) that are executed on host CPU and code sections (known as **device kernels** and typically computation workload) that are dispatched to be executed on **SYCL devices** (CPU, GPU, FPGA and other). 

A SYCL host code uses **SYCL constructs and classes** to organize the parallel computation in heterogenous system and **SYCL-aware C++ compiler** will translate the SYCL source code into respective binaries targeting the underlying devices delivering overall good performance through parallel data computation.

**oneAPI DPC++/C++ Compiler** is Intel® distribution of SYCL-aware C++ compiler.


### **Overview of SYCL C++ Program**

![SYCL C++ Program Skeleton](./images/sycl-cpp-skeleton.jpg)

### **Exercise**

#### 1) Verify availability of SYCL devices

In [ ]:
# List SYCL devices on the system
! sycl-ls

In [ ]:
# Check the DPC++ Compiler version
! icx --version

In [ ]:
# Check SYCLomatic tool version
! c2s --version

#### 2) Prepare CUDA source code

In [ ]:
%%writefile vectoradd.cu

//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <cuda.h>
#include <iostream>
#include <vector>
#define N 16

//# kernel code to perform VectorAdd on GPU
__global__ void VectorAddKernel(float* A, float* B, float* C)
{
        C[threadIdx.x] = A[threadIdx.x] + B[threadIdx.x];
}

int main()
{
        //# Initialize vectors on host
        float A[N] = {1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1};
        float B[N] = {2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2};
        float C[N] = {0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0};

        //# Allocate memory on device
        float *d_A, *d_B, *d_C;
        cudaMalloc(&d_A, N*sizeof(float));
        cudaMalloc(&d_B, N*sizeof(float));
        cudaMalloc(&d_C, N*sizeof(float));

        //# copy vector data from host to device
        cudaMemcpy(d_A, A, N*sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_B, B, N*sizeof(float), cudaMemcpyHostToDevice);

        //# sumbit task to compute VectorAdd on device
        VectorAddKernel<<<1, N>>>(d_A, d_B, d_C);

        //# copy result of vector data from device to host
        cudaMemcpy(C, d_C, N*sizeof(float), cudaMemcpyDeviceToHost);

        //# print result on host
        for (int i = 0; i < N; i++) std::cout<< C[i] << " ";
        std::cout << "\n";

        //# free allocation on device
        cudaFree(d_A);
        cudaFree(d_B);
        cudaFree(d_C);
        return 0;
}

**Information:**
* **VectorAddKernel() is CUDA kernel code** that performs parallelized addition of A + B and stores the result to C.
* Note that the addition operation of these A+B values can be performed independently.
* **float A[N], B[N], C[N]** are floating point arrays on **CPU (host) memory**.
* **float *d_A, *d_B, *d_C** are floating point pointers to **GPU (device) memory**.
* GPU device memory is allocated using **cudaMalloc()**.
* The floating point values on **CPU memory (A[N] and B[N]) are copied to GPU memory** at location pointed by d_A and d_B pointers using **cudaMemcpy()**. 
* **VectorAddKernel<<<1,N>>>(d_A, d_B, d_C) is how CUDA kernel is submitted to GPU device** in parallelized fashion. In CUDA program, the parallelized addition of "A+B" is offloaded into GPU execution units that are executed simulateneously.
* The **result of the addition in GPU device memory (d_C) is copied to CPU memory (C[N])**.
* Finally, the device memory allocated earlier (d_A, d_B and d_C) are freed.
* The CUDA compiler understands CUDA-specific constructs (e.g, <<<1,N>>> and \_\_global\_\_) and APIs (e.g. cudaMalloc(), cudaMemcpy() and cudaFree()).


#### 3) Use SYCLomatics tool to convert CUDA code to SYCL C++

In [ ]:
! c2s vectoradd.cu --cuda-include-path=/usr/local/cuda-12.1/include --out-root=sycl_output --gen-helper-function

**Information:**
* --cuda-include-path=<path to CUDA include>: Specify the CUDA include header path.
* --out-root=<SYCL output directory>: Specify the SYCL code output
* --gen-helper-function : Generate SYCLomatic helper header files to output 

**Note:**
* oneAPI Base Toolkit version 2023.02 supports CUDA Toolkit version 12.1.

#### 4) Review the SYCL output 

In [ ]:
! tree sycl_output

**Information:**
* MainSourceFile.yaml – CUDA to SYCL conversion log
* vectoradd.dp.cpp – the converted SYCL C++ source code
* include/ – SYCLomatic generated SYCL helper headers

In [ ]:
# Check the converted CUDA converted code (in SYCL C++ code) 
! cat sycl_output/vectoradd.dp.cpp

**Information:**

![SYCL C++ Conversion 1](./images/sycl-convert-1.jpg)
![SYCL C++ Conversion 1](./images/sycl-convert-2.jpg)


#### 5) Edit SYCL code to print SYCL device information

Add following code to sycl_output/vectoradd.dp.cpp as indicated by '+' as follow:
```c
        //# copy result of vector data from device to host
        q_ct1.memcpy(C, d_C, N * sizeof(float)).wait();

+        // Print SYCL-device info
+        std::cout << "Running on device: "
+                  << q_ct1.get_device().get_info<sycl::info::device::name>() << "\n";
        
        //# print result on host
        for (int i = 0; i < N; i++) std::cout<< C[i] << " ";
        std::cout << "\n";
```
**Note:**
* From Jupyter Notebook, navigate to the file from the LHS file explorer, double-click on the selected file.
* Edit the file as shown below and save the change by Ctrl+s.

#### 6) Compile SYCL code using DPC++ compiler

In [ ]:
! cd sycl_output && icpx -fsycl -I include vectoradd.dp.cpp -o vectoradd_prog

#### 7) Run the executable

In [ ]:
! cd sycl_output && file vectoradd_prog

In [ ]:
! cd sycl_output && ./vectoradd_prog

#### 8) Running SYCL program on different SYCL devices

The execution of SYCL device/kernel code can be influenced by using **ONEAPI_DEVICE_SELECTOR=\<option\>**.
The value for the \<option\> can be obtained from the output of 'sycl-ls' command.

**Note:** For more information about ONEAPI_DEVICE_SELECTOR environment variable, please refer to https://intel.github.io/llvm-docs/EnvironmentVariables.html#sycl-device-filter

In [ ]:
! sycl-ls

In [ ]:
! cd sycl_output && ONEAPI_DEVICE_SELECTOR=opencl:gpu ./vectoradd_prog

In [ ]:
! cd sycl_output && ONEAPI_DEVICE_SELECTOR=opencl:cpu ./vectoradd_prog

In [ ]:
! cd sycl_output && ONEAPI_DEVICE_SELECTOR=opencl:cpu,gpu ./vectoradd_prog

**Information:**
* If ONEAPI_DEVICE_SELECTOR environment variable is not used, SYCL program will default to SYCL device that gives the best computation performance.
* Setting ONEAPI_DEVICE_SELECTOR to a specific value limits the SYCL program to be executed on the specified SYCL device.
* For ONEAPI_DEVICE_SELECTOR=opencl:cpu,gpu, SYCL program may be executed on CPU if GPU device is absent.

### **Conclusion:**

* CUDA program runs on NVIDIA GPU in parallel fashion over a pool of Execution Units (Streaming Multi-processor).
* SYCL C++ program can run in CPU, GPU, FPGA and other accelerator.
* The conversion from CUDA to SYCL C++ source code can be accelerated using SYCLomatic tool.
* A SYCL C++ code is then compiled using Intel oneAPI DPC++/C++ Compiler.
* A specific version of SYCLomatic tool (bundled in Intel oneAPI Base Toolkit) support up-to a specific CUDA Toolkit version. (Lagging)

**Notices & Disclaimers** 

Intel technologies may require enabled hardware, software or service activation. 

No product or component can be absolutely secure.  

Your costs and results may vary.  

No license (express or implied, by estoppel or otherwise) to any intellectual property rights is granted by this document, with the sole exception that code included in this document is licensed subject to the Zero-Clause BSD open source license (0BSD), [Open Source Initiative](https://opensource.org/licenses/0BSD). No rights are granted to create modifications or derivatives of this document. 

© Intel Corporation.  Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries.  Other names and brands may be claimed as the property of others.  